In [8]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [9]:
full_targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")
full_mtl = Table.read("strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits")

ii = np.in1d(full_mtl['TARGETID'], zcat['TARGETID'])
in_mtl = full_mtl[ii]

ii = np.in1d(full_targets['TARGETID'], zcat['TARGETID'])
in_targets = full_targets[ii]

print(len(zcat), len(in_mtl), len(in_targets))
in_mtl.sort(keys='TARGETID')
zcat.sort(keys='TARGETID')
in_targets.sort(keys='TARGETID')

864670 864670 864670


In [10]:
assert np.all(in_mtl['TARGETID']==zcat['TARGETID'])
assert np.all(in_mtl['TARGETID']==in_targets['TARGETID'])

In [11]:
zcat_large = zcat[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_mtl_large = in_mtl[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_targets_large = in_targets[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]

In [12]:
names = {}
for i in range(len(in_mtl_large)):
    a = desi_mask.names(in_mtl_large['DESI_TARGET'][i])
    n = ''
    for aa in a:
        n = n +' '+ aa
    if n not in names.keys():
        names[n] = 1
    else:
        names[n] += 1

In [13]:
names

{' QSO QSO_SOUTH BGS_ANY MWS_ANY': 520,
 ' ELG QSO ELG_SOUTH QSO_SOUTH': 4447,
 ' QSO QSO_SOUTH BGS_ANY': 376,
 ' QSO QSO_SOUTH': 2677,
 ' QSO QSO_SOUTH MWS_ANY': 49,
 ' ELG QSO ELG_SOUTH QSO_SOUTH BGS_ANY': 55,
 ' QSO QSO_SOUTH STD_WD BGS_ANY MWS_ANY': 8,
 ' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY': 4,
 ' LRG QSO LRG_1PASS LRG_SOUTH QSO_SOUTH LRG_1PASS_SOUTH': 2,
 ' QSO QSO_SOUTH STD_WD MWS_ANY': 1}